# This is a project for using the basic functionality of ifcopenshell

This is a solution proposal for the 03 basic project. 

Main Challenge: 
**Filter out all load bearing walls and visualize them**

Optional:
**Pull other properties and dump it to a csv file.**

Main Challenge help: 
* A prerequisit for this challenge is the **01 Basic Introduction to Python for use with BIM notebook**
* The **02 Basics with ifcopenshell notebook** might also be of use. 

And like always in programming there is **google** and especially for opensource community work, like with ifcopenshell, you could ask the community on github. 

Note: In all cells below. Add code after the comments. 

In [6]:
# Importing relevant modules
import ifcopenshell
import ifcopenshell.geom
from ifc_viewer import ifc_viewer

## Main Challenge

In [7]:
# Create a function that takes in a tuple of IfcWall objects and returns the ones that are loadbaring in a list
# Use the "." operator to travers and get its property sets. It is useful to check the buildingSMART documentation on IfcWall
# Note: if you have a property in a variable p, then p.NominalValue.wrappedValue will give you the value.

def get_load_bearing(walls):
    def is_load_bearing(pset_wall_common):
        props = pset_wall_common.HasProperties
        for p in props:
            if p.Name =="LoadBearing":
                return p.NominalValue.wrappedValue
    # Thomas Krijens pro-tip: instead of appending to a list you could consider using a yield statement
    lb_walls = []
    for wall in walls:
        psets = [rel.RelatingPropertyDefinition for rel in wall.IsDefinedBy if rel.is_a("IfcRelDefinesByProperties")]
        pset_wall_commons = [pset for pset in psets if pset.Name == "Pset_WallCommon"]
        if len(pset_wall_commons) is not 0:
            if is_load_bearing(pset_wall_commons[0]):
                lb_walls.append(wall)
        else:
            print("Wall %s has no Pset_WallCommon!!"%(wall.Name))
            
    return lb_walls



In [8]:
# Start with importing your chosen model. Store it in a variable. 
file = ifcopenshell.open("../models/Grethes_hus_bok_2.ifc")
# Get a list of all walls and store it in a variable. 
walls = file.by_type("IfcWall")
# Use the function you have created, giving it the variable having the walls and store the result in another variable. 
load_bearing_walls = get_load_bearing(walls)

In [9]:
## Set the proper geometry settings 
s = ifcopenshell.geom.settings()
s.set(s.USE_PYTHON_OPENCASCADE, True)
## Create a viewer object
viewer = ifc_viewer()
## Run trhough the loarbaring walls and create shapes and add that to the viewer object using the DisplayShape function. 
for wall in load_bearing_walls:
    shape = ifcopenshell.geom.create_shape(s, wall)
    viewer.DisplayShape(wall, shape.geometry, shape.styles)
## Have the viewer display inline using the Display() function.
viewer.Display()

Renderer(background='white', camera=PerspectiveCamera(children=(DirectionalLight(intensity=0.5, position=(50.0…

HTML(value='No element selected')